Давайте построим простую классификацию для пассажиров Титаника

По данным пассажира будет пытаться угадать, выжил он или нет

В этом ноутбуке сделаем предобработку тренировочных данных, а построение классификатора и вычисление метрик качества на тестовых данных будут домашним заданием

Вам понадобятся датасеты Titanic_train.csv и Titanic_test.csv

In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

In [2]:
# Считаем тренировочный датасет
df = pd.read_csv('Titanic_train.csv')

In [3]:
# посмотрим, как обращаться к колонкам датасета
df.columns

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

В первом блоке анализа мы посмотрим, какие у нас есть данные, какие данные пропущены

In [4]:
# полюбуемся на фрагмент датасета
df.sample(5)

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
47,375,376,1,1,"Meyer, Mrs. Edgar Joseph (Leila Saks)",female,NaN,1,0,PC 17604,82.1708,NaN,C
102,122,123,0,2,"Nasser, Mr. Nicholas",male,32.5,1,0,237736,30.0708,NaN,C
71,767,768,0,3,"Mangan, Miss. Mary",female,30.5,0,0,364850,7.7500,NaN,Q
563,55,56,1,1,"Woolner, Mr. Hugh",male,NaN,0,0,19947,35.5000,C52,S
476,10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S


In [5]:
# выясним, в каких столбцах не хватает данных
df.isna().sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            143
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          552
Embarked         2
dtype: int64

Данных не хватает в столбцах Age, Cabin, Embarked:
    1. У 143 пассажиров не известен возраст
    2. у 552 пассажиров не известен номер каюты
    3. У 2 пассажиров не известен порт посадки

In [6]:
# Недостающие данные по возрасту заполним средним арифметическим
mean_age = df['Age'].mean() # здесь вычисляем
print ('средний возраст пассажира: ',mean_age)
df['Age'].fillna(mean_age , inplace=True) # здесь заполняем

средний возраст пассажира:  29.539402460456945


In [7]:
# Посмотрим, из какого порта отправилось большинство пассажиров
# будет логично предположить, что 2 пассажира с неизвестным портом отправления сели там же
print(df['Embarked'].value_counts())

S    514
C    128
Q     68
Name: Embarked, dtype: int64


In [8]:
# Больше всего пассажиров отправились в путь на Титанике из порта S - Southampton
# Назначим это значение пассажирам, чей порт посадки не известен
df['Embarked'].fillna('S',inplace=True)

В столбцах Unnamed: 0' и 'PassengerId содержатся порядковые номера пассажиров, они нам не нужны

'Cabin' содержит слишком много пропущенных значений, заполнять их бессмысленно, хотя
из этих данных, вернее, из их наличия можно было бы извлечь полезную информацию: единственный список с номерами кают есть был найден только для пассажиров 1 класса , а вот у пассажиров 2 и 3 класса наличие номера каюты означает, что этот пассажир, скорее всего выжил. Это может стать точкой роста для классификатора, но сейчас мы все равно избавимся от этого столбца

Из 'Name' и 'Ticket' (идентификатор билета) тоже можно вытащить полезную информацию, однако сейчас опять же эти данные рассматривать не будем

In [9]:
# удаляем лишние столбцы из датасета
del df['Unnamed: 0']
del df['PassengerId']
del df['Name']
del df['Cabin']
del df['Ticket']

In [10]:
#  сделаем предобработку данных по полу и порту посадки
# они заданы как категории, нам же для обучения нужны числа
# есть разные способы превратить категории в числа, один из них 
# - задать числовые категории по словарю

# создаем словари
dict_Sex = {'male': 1, 'female': 0}
dict_Embarked = {'S': 0, 'C': 1, 'Q':2}

# преобразуем данные при помощи map
df['Sex'] = df['Sex'].map(dict_Sex)
df['Embarked'] = df['Embarked'].map(dict_Embarked)



In [11]:
# полюбуемся на то, что у нас получилось
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,47.000000,0,0,9.0000,0
1,1,3,1,29.539402,0,0,7.7500,2
2,0,2,1,34.000000,1,0,26.0000,0
3,1,1,0,44.000000,0,1,57.9792,1
4,0,3,0,47.000000,1,0,14.5000,0


In [12]:
# разделим данные на признаки и целевую переменную
x_train = df.drop(['Survived'], axis = 1)
y_train = df[['Survived']]

Ваше задание:
1. Сделать точно такую же предобработку тестовых данных. Подумайте, на какие значения будет корректно заменять пропуски в графах "возраст" и "порт посадки" в тестовой выборке.
2. Обучить НБК. Какую модель вы выберете?
3. Вычислить confusion matrix на тестовых данных
4. Вычислить метрики качества модели
5. Подумайте, каких пассажиров модель классифицирует лучше - выживших или не выживших
6. Подумайте, как проинтерпретировать ошибку второго рода

In [13]:
sum(df['Survived'])/len(df)

0.37780898876404495

In [14]:
df[df['Survived'] == 1]['Sex'].value_counts().iloc[1]/sum(df['Survived'])

0.3159851301115242

In [15]:
df[df['Survived'] == 1]['Sex'].value_counts().iloc[0]/sum(df['Survived'])

0.6840148698884758

In [16]:
sum(df[df['Sex'] == 1]['Survived'])/sum(df['Sex'])

0.1827956989247312

In [17]:
vc = df[df['Sex'] == 0]['Survived'].value_counts()
vc[1]/(vc[1]+vc[0])

0.7449392712550608

In [18]:
# Считаем тренировочный датасет
df_test = pd.read_csv('Titanic_test.csv')

In [19]:
# посмотрим, как обращаться к колонкам датасета
df_test.columns

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

В первом блоке анализа мы посмотрим, какие у нас есть данные, какие данные пропущены

In [20]:
# полюбуемся на фрагмент датасета
df_test.sample(5)

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
54,103,104,0,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
104,766,767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6000,NaN,C
80,346,347,1,2,"Smith, Miss. Marion Elsie",female,40.0,0,0,31418,13.0000,NaN,S
165,505,506,0,1,"Penasco y Castellana, Mr. Victor de Satode",male,18.0,1,0,PC 17758,108.9000,C65,C
145,564,565,0,3,"Meanwell, Miss. (Marion Ogden)",female,NaN,0,0,SOTON/O.Q. 392087,8.0500,NaN,S


In [21]:
# выясним, в каких столбцах не хватает данных
df_test.isna().sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             34
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          135
Embarked         0
dtype: int64

Данных не хватает в столбцах Age, Cabin, Embarked:
    1. У 143 пассажиров не известен возраст
    2. у 552 пассажиров не известен номер каюты
    3. У 2 пассажиров не известен порт посадки

In [22]:
# Недостающие данные по возрасту заполним средним арифметическим из тренировочной выборки
# mean_age = df['Age'].mean() # здесь вычисляем
print ('средний возраст пассажира: ',mean_age)
df_test['Age'].fillna(mean_age , inplace=True) # здесь заполняем

средний возраст пассажира:  29.539402460456945


In [23]:
# Посмотрим, из какого порта отправилось большинство пассажиров
# будет логично предположить, что 2 пассажира с неизвестным портом отправления сели там же
print(df_test['Embarked'].value_counts())

S    130
C     40
Q      9
Name: Embarked, dtype: int64


In [24]:
# Больше всего пассажиров отправились в путь на Титанике из порта S - Southampton
# Назначим это значение пассажирам, чей порт посадки не известен
df_test['Embarked'].fillna('S',inplace=True)

В столбцах Unnamed: 0' и 'PassengerId содержатся порядковые номера пассажиров, они нам не нужны

'Cabin' содержит слишком много пропущенных значений, заполнять их бессмысленно, хотя
из этих данных, вернее, из их наличия можно было бы извлечь полезную информацию: единственный список с номерами кают есть был найден только для пассажиров 1 класса , а вот у пассажиров 2 и 3 класса наличие номера каюты означает, что этот пассажир, скорее всего выжил. Это может стать точкой роста для классификатора, но сейчас мы все равно избавимся от этого столбца

Из 'Name' и 'Ticket' (идентификатор билета) тоже можно вытащить полезную информацию, однако сейчас опять же эти данные рассматривать не будем

In [25]:
# удаляем лишние столбцы из датасета
del df_test['Unnamed: 0']
del df_test['PassengerId']
del df_test['Name']
del df_test['Cabin']
del df_test['Ticket']

In [26]:
# Преобразуем данные при помощи map по словарям, подготовленным ранее для обучающей выборки
df_test['Sex'] = df_test['Sex'].map(dict_Sex)
df_test['Embarked'] = df_test['Embarked'].map(dict_Embarked)

In [27]:
# полюбуемся на то, что у нас получилось
df_test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,32.0,0,0,7.7500,2
1,0,1,0,25.0,1,2,151.5500,0
2,1,3,1,20.0,1,0,7.9250,0
3,1,3,1,3.0,4,2,31.3875,0
4,1,1,0,38.0,0,0,227.5250,1


In [28]:
# разделим данные на признаки и целевую переменную
x_test = df_test.drop(['Survived'], axis = 1)
y_test = df_test[['Survived']]

In [29]:
mnb = GaussianNB()

In [30]:
# обучаем модель
mnb.fit(x_train, y_train.values.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [31]:
# делаем предсказания на тестовой выборке
y_pred = mnb.predict(x_test)
y_pred

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0])

In [32]:
# смотрим, где угадали и не угадали
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

array([[88, 18],
       [22, 51]])

In [33]:
# строим confusion matrix - таблицу правильных и неправильных предсказаний
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive

Ac = mnb.score(x_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)

print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Precision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.776536312849162
Sensitivity:  0.6986301369863014
Specificity:  0.8301886792452831
Precision:  0.7391304347826086
Type I error rate:  0.16981132075471697
Type II error rate:  0.3013698630136986
